# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../images/ClassLocation.jpg)

# Create a model with Advanced Packages - Part 2

Load model from Part 1 and amend with UZF & MVR, rerun and look at the MVR budget

In [ ]:
import warnings
from pathlib import Path

import flopy
import matplotlib.pyplot as plt
import numpy as np

warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)

## Load the model amended with UZF and MVR

In [ ]:
name = "ad-p1"
ws = Path(f"./temp/{name}")

sim = flopy.mf6.MFSimulation.load(
    sim_name=name,
    exe_name="mf6",
    sim_ws=ws,
)

### Get the GWF model

In [ ]:
gwf = sim.get_model("ad-p1")

# get some needed parameter values
nlay = gwf.dis.nlay.array

### Add a diversion within the SFR package
For simplicity, diversion will run due north away from the existing stream that drains the lake.  The diversion will traverse 4 cells all in column 14 (0-based)

In [ ]:
# Original data
bed_elev = [142.0, 141.5, 141.0, 140.5, 140.0] + [140.4, 140.3, 140.2, 140.1]
upstfr = [0.0, 1.0, 1.0, 1.0, 1.0] + [0.0, 1.0, 1.0, 1.0]
nconn = [1, 2, 2, 3, 1] + [2, 2, 2, 1]
ndiv = [0, 0, 0, 1, 0] + [0, 0, 0, 0]
nreaches = len(bed_elev)

In [ ]:
# <ifno> <cellid(ncelldim)> <rlen> <rwid> <rgrd> <rtp> <rbth> <rhk> <man> <ncon> <ustrf> <ndv>
sfr_pakdata = []
ct = 0
divCol = 14  # 0-based
for idx in range(nreaches):
    rchlen = 1000.0
    if idx <= 4:
        cellid = (0, 8, 11 + idx)
    else:
        ct += 1
        cellid = (0, 8 - ct, divCol)
        if idx <= 6:
            rchlen = 500.0

    sfr_pakdata.append(
        (
            idx,
            cellid,
            rchlen,
            1.0,
            1e-3,
            bed_elev[idx],
            0.1,
            1.0,
            0.035,
            nconn[idx],
            upstfr[idx],
            ndiv[idx],
            "sfr",
        )
    )

sfr_pakdata

In [ ]:
sfr_conn = [
    [0, -1],
    [1, 0, -2],
    [2, 1, -3],
    [3, 2, -4, -5],
    [4, 3],
    [5, 3, -6],
    [6, 5, -7],
    [7, 6, -8],
    [8, 7],
]
sfr_conn

In [ ]:
obs_file = f"{name}.sfr.obs"
csv_file = obs_file + ".csv"
obs_dict = {
    obs_file + ".csv": [
        ("inflow_rch4", "inflow", (3,)),
        ("outflow_rch4", "outflow", (3,)),
        ("inflow_rch5", "inflow", (4,)),
        ("divertedAmt", "inflow", (5,)),
    ]
}

#### Define the `DIVERSIONS` block
Use the `CPRIOR` option `EXCESS`: <br>
<br>
`EXCESS`: a diversion is made only if $Q_{DS}$ for reach `IFNO` exceeds the value of `DIVFLOW`. If this occurs, then the quantity of water diverted is
the excess flow ($Q_{DS}$ − DIVFLOW) and $Q_{DS}$ from reach IFNO is set equal to DIVFLOW. This represents a flood-control type of diversion

In [ ]:
#      <ifno> <idv> <iconr> <cprior>
divs = [3, 0, 5, "EXCESS"]

#### Define the stress period data
Because we added a diversion, need to set the transient data

In [ ]:
excess_amt = 10000.0
sfr_spd = {0: [3, "DIVERSION", 0, excess_amt]}

#### Redefine the SFR package
There is one gotcha here.  In the script that follows, if we don't specify the argument `pname`, FloPy will append the package to the existing `MODFLOW 6` simulation rather than replace the existing SFR package with the new one that includes the diversion.

In [ ]:
sfr = flopy.mf6.ModflowGwfsfr(
    gwf,
    mover=True,
    boundnames=True,
    print_input=True,
    print_flows=True,
    print_stage=True,
    length_conversion=3.28081,
    time_conversion=86400.0,
    nreaches=nreaches,
    packagedata=sfr_pakdata,
    connectiondata=sfr_conn,
    diversions=divs,
    observations=obs_dict,
    perioddata=sfr_spd,
    pname="sfr_0",
)

## Write the simulation

In [ ]:
sim.write_simulation()

## Run the simulation

In [ ]:
sim.run_simulation()

## Have another look at the output

In [ ]:
extents = (0.0, gwf.dis.delr.array.sum(), 0.0, gwf.dis.delc.array.sum())

# load the observations
lak_results = gwf.lak.output.obs().data
sfr_results = gwf.sfr.output.obs().data
gwf_results = gwf.obs[0].output.obs().data

# Figure properties
figure_size = (6.3, 5.6)
masked_values = (0, 1e30, -1e30)

# create MODFLOW 6 head object
hobj = gwf.output.head()

# create MODFLOW 6 cell-by-cell budget object
cobj = gwf.output.budget()

kstpkper = hobj.get_kstpkper()

head = hobj.get_data(kstpkper=kstpkper[0])
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(
    cobj.get_data(text="DATA-SPDIS", kstpkper=kstpkper[0])[0],
    gwf,
)

# add lake stage to heads
head[head == 1e30] = lak_results["STAGE"][-1]

# observation locations
xcenters, ycenters = gwf.modelgrid.xycenters[0], gwf.modelgrid.xycenters[1]
p1 = (xcenters[3], ycenters[3])
p2 = (xcenters[13], ycenters[13])

shape3d = (gwf.dis.nlay.array, gwf.dis.nrow.array, gwf.dis.ncol.array)

In [ ]:
fig, axd = plt.subplot_mosaic(
    [
        ["a"],
        ["a"],
        ["b"],
    ],
    layout="constrained",
    figsize=(4, 6.9),
)

ax = axd["a"]
mm = flopy.plot.PlotMapView(gwf, ax=ax, extent=extents)
mm.plot_bc("CHD", color="cyan")
mm.plot_bc("SFR", color="blue", alpha=0.1)
mm.plot_bc("UZF", color="lightgreen", alpha=0.7)
mm.plot_inactive(color_noflow="#5DBB63")
mm.plot_grid(lw=0.5, color="black")
cv = mm.contour_array(
    head,
    levels=np.arange(140, 160, 2),
    linewidths=0.75,
    linestyles="-",
    colors="blue",
)
plt.clabel(cv, fmt="%1.0f")
mm.plot_vector(qx, qy, normalize=True, color="0.75")
ax.plot(p1[0], p1[1], marker="o", mfc="red", mec="black", ms=4)
ax.annotate("Point A", (p1[0] + 150, p1[1]))
ax.plot(p2[0], p2[1], marker="o", mfc="red", mec="black", ms=4)
ax.annotate("Point B", (p2[0] + 150, p2[1]))
ax.plot(p2[0], p1[1], marker="o", mfc="yellow", mec="purple", ms=4)
ax.annotate("MAW", (p2[0] - 1500, p1[1] + 150))
ax.annotate("UZF", (10.75e3, 11e3))
ax.set_xlabel("x-coordinate, in feet")
ax.set_ylabel("y-coordinate, in feet")

ax = axd["b"]
xs = flopy.plot.PlotCrossSection(gwf, ax=ax, line={"row": 8})
xs.plot_array(np.ones(shape3d), head=head, cmap="jet")
xs.plot_bc("CHD", color="cyan", head=head)
xs.plot_ibound(color_noflow="#5DBB63", head=head)
xs.plot_grid(lw=0.5, color="black")
ax.set_xlabel("x-coordinate, in feet")
ax.set_ylim(67, 160)
ax.set_ylabel("Elevation, in feet")

plt.show(block=False)

In [ ]:
sfr_results = gwf.sfr.output.obs().data
sfr_results

In [ ]:
dtype = [
    ("time", float),
    ("INFLOW_RCH4", float),
    ("OUTFLOW_RCH4", float),
    ("INFLOW_RCH5", float),
    ("DIVERTEDAMT", float),
]

results = np.zeros((sfr_results.shape[0] + 1), dtype=dtype)
results["time"][1:] = sfr_results["totim"]
results["INFLOW_RCH4"][1:] = sfr_results["INFLOW_RCH4"]
results["OUTFLOW_RCH4"][1:] = sfr_results["OUTFLOW_RCH4"]
results["INFLOW_RCH5"][1:] = sfr_results["INFLOW_RCH5"]
results["DIVERTEDAMT"][1:] = sfr_results["DIVERTEDAMT"]
results

In [ ]:
# create the figure
fig, ax = plt.subplots(
    ncols=1,
    nrows=1,
    sharex=True,
    figsize=(6.3, 3.15),
    constrained_layout=True,
)

ax.set_xlim(0, 3000)
# ax.set_ylim(110, 160)
ax.plot(
    results["time"],
    results["INFLOW_RCH4"],
    lw=0.75,
    ls="--",
    color="black",
    label="Reach 4 inflow",
)
ax.plot(
    results["time"],
    results["OUTFLOW_RCH4"],
    lw=0.75,
    ls=":",
    color="red",
    label="Reach 4 outflow",
)
ax.plot(
    results["time"],
    results["INFLOW_RCH5"],
    lw=1.0,
    ls="-",
    color="green",
    label="Reach 5 inflow",
)
ax.plot(
    results["time"],
    results["DIVERTEDAMT"],
    lw=1.0,
    ls="-",
    color="blue",
    label="Diversion",
)
ax.axhline(y=0, color="black")
ax.set_xlabel("Simulation time, in days")
ax.set_ylabel("Flow, in $ft^3$ per day")
plt.legend(loc="lower left")

plt.show(block=False)

### Update MVR package